乳腺癌数据的logistic回归

In [71]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import load_breast_cancer
import torchvision.datasets as datasets
from torch.utils.data import Dataset,DataLoader

自定义dataset

In [82]:
class Breast_CancerDataset(Dataset):
    def __init__(self,train=True):
        X,y=load_breast_cancer(return_X_y=True)
        if train:
            self.X=torch.tensor(X[0:427,:]).float()
            self.y=torch.tensor(y[0:427]).float()
        else:
            self.X=torch.tensor(X[427:,:]).float()
            self.y=torch.tensor(y[427:]).float()
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx]

In [73]:
tra=Breast_CancerDataset(train=True)
tes=Breast_CancerDataset(train=False)
tra_dloader=DataLoader(tra,batch_size=427,shuffle=True)
tes_dloader=DataLoader(tes,batch_size=142,shuffle=True)

建立神经网络

In [79]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fun = nn.Linear(30,1)
        self.activate=nn.Sigmoid()
        self.flatten = nn.Flatten(0,1)
    def forward(self, x):
        x = self.fun(x)
        x =self.activate(x)
        return self.flatten(x)

损失函数与优化器

In [86]:
loss_fun=nn.BCELoss()
net=Net()
optimizer = torch.optim.Adam(net.parameters())

网络训练

In [87]:
epoch=10000
for i in range(0,epoch):
    for idx,data in enumerate(tra_dloader):
        optimizer.zero_grad()
        x=data[0]
        target=data[1]
        y=net(x)
        loss=loss_fun(y,target)
        loss.backward()
        optimizer.step()
print(loss)        

tensor(0.0659, grad_fn=<BinaryCrossEntropyBackward>)


计算测试误差

In [88]:
for idx,data in enumerate(tes_dloader):
    x=data[0]
    target=data[1]
    y=net(x)
    tes_loss=loss_fun(y,target)
    print(tes_loss)

tensor(0.1217, grad_fn=<BinaryCrossEntropyBackward>)


计算预测正确率

In [93]:
fun2=nn.L1Loss()
for idx,data in enumerate(tes_dloader):
    x=data[0]
    target=data[1]
    y=net(x)
    pre=torch.round(y)
    print(1-fun2(y,target))

tensor(0.9285, grad_fn=<RsubBackward1>)
